In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [67]:
def u(x):
    return np.log(1 + x * x) + np.sin(x)

In [123]:
class Solver:

    def __init__(self, n, CFL, u):
        self.L = 1
        self.T = 1
        self.h = 1 / (n - 1)
        self.CFL = CFL
        self.u = u

        self.tau = self.CFL * self.h

        self.NX = int(self.L / self.h + 1)
        self.TX = int(self.T / self.tau + 1) 

        self.x_i = np.array([(i - 1) * self.h for i in range(1, self.NX + 1)])
        self.t_i = np.array([(i - 1) * self.tau for i in range(1, self.TX + 1)])

        self.u_n_m = self.u(self.x_i)
        self.u_next_m = self.u_n_m.copy()

        self.analytic_x_i = np.linspace(0, 1, 11)
        self.analytic_u = np.sin(self.analytic_x_i) + np.log(1 + np.power((self.analytic_x_i - 1), 2))
        self.analytic_all = np.sin(self.x_i) + np.log(1 + np.power((self.x_i - 1), 2))

        self.u_result = []
        self.delta_u = []

    def solve(self):
        for t in self.t_i[1:]:
            self.u_next_m[0] = np.log(1 + t * t)
            self.u_next_m[1] = self.u_next_m[0] + self.h * (1 - (2*t)/(1+t*t)) + (np.power(self.h, 2) / 2) * ((2*(1 - t*t))/np.power((t*t + 1), 2)) + (np.power(self.h, 3) / 6) * (- 1 - (4*t*(np.power(t,2) - 3))/(np.power((1 + t*t), 3)))
            self.u_next_m[2] = self.u_next_m[0] + 2 * self.h * (1 - (2*t)/(1+t*t)) + 4 * (np.power(self.h, 2) / 2) * ((2*(1 - t*t))/np.power((t*t + 1), 2)) + 8 * (np.power(self.h, 3) / 6) * (- 1 - (4*t*(np.power(t,2) - 3))/(np.power((1 + t*t), 3)))
            for i in range(3, self.NX):
                self.u_next_m[i] = self.u_n_m[i] \
                        + (self.CFL / 6) * (2*self.u_n_m[i-3] - 9*self.u_n_m[i-2] + 18*self.u_n_m[i-1] - 11*self.u_n_m[i]) \
                        + (np.power(self.CFL, 2) / 2) * (-self.u_n_m[i-3] + 4*self.u_n_m[i-2] - 5*self.u_n_m[i-1] + 2*self.u_n_m[i]) \
                        - (np.power(self.CFL, 3) / 6) * (-self.u_n_m[i-3] + 3*self.u_n_m[i-2] - 3*self.u_n_m[i-1] + self.u_n_m[i]) \
                        + self.tau*np.cos(self.x_i[i]) + 0.5*np.power(self.tau, 2)*np.sin(self.x_i[i]) - (np.power(self.tau, 3)/6)*np.cos(self.x_i[i])
            self.u_n_m = self.u_next_m.copy()
    def display(self):
        for ind, x in enumerate(self.analytic_x_i):
            i, = np.where(np.isclose(self.x_i, x))
            self.u_result.append(float(self.u_n_m[i]))
            self.delta_u.append(np.abs(self.u_result[ind] - self.analytic_u[ind]))
        self.df = pd.DataFrame({"x": self.analytic_x_i, "u аналит.": self.analytic_u, "u числ. ": self.u_result ,"delta u": self.delta_u})
        pd.options.display.float_format = '{:.6E}'.format
        display(self.df)
        max_err = self.df['delta u'].max()
        print(f'{max_err}')

    def displayMaxErr(self):
        print(f'cfl = {self.CFL}\tmax error = {np.max(np.abs(self.analytic_all - self.u_n_m))}')

In [102]:
for cfl in [0.9, 1., 1.1, 1.9, 2., 2.1, 2.9, 3., 3.1]:
    s = Solver(1001, cfl, u)
    s.solve()
    s.displayMaxErr()

cfl = 0.9	max error = 1.0125269077717793e+23
cfl = 1.0	max error = 2.7166602301065268e-11
cfl = 1.1	max error = 9.999999983334451e-05
cfl = 1.9	max error = 0.0005999999639838549
cfl = 2.0	max error = 1.6141021852433823e-10
cfl = 2.1	max error = 18256069396.093132
cfl = 2.9	max error = 0.002399997691848954
cfl = 3.0	max error = 0.0009999998332083093
cfl = 3.1	max error = 2.390531471596097e+63


In [124]:
s_r = Solver(21, 1.5, u)
s_r.solve()
s_r.display()

/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_79727/2276887736.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.u_result.append(float(self.u_n_m[i]))


,x,u аналит.,u числ.,delta u
0,0.000000E+00,6.931472E-01,6.681498E-01,2.499735E-02
1,1.000000E-01,6.931603E-01,6.683245E-01,2.483581E-02
2,2.000000E-01,6.933656E-01,6.690499E-01,2.431569E-02
3,3.000000E-01,6.942963E-01,6.709415E-01,2.335480E-02
4,4.000000E-01,6.969030E-01,6.750541E-01,2.184890E-02
5,5.000000E-01,7.025691E-01,6.828680E-01,1.970108E-02
6,6.000000E-01,7.130625E-01,6.962201E-01,1.684240E-02
7,7.000000E-01,7.303954E-01,7.171362E-01,1.325918E-02
8,8.000000E-01,7.565768E-01,7.475465E-01,9.030274E-03
9,9.000000E-01,7.932772E-01,7.890136E-01,4.263670E-03


0.024997346517038443


In [108]:
for h in [101, 201, 401]:
    s = Solver(h, 1, u)
    s.solve()
    print(h={h})
    s.display()

101


/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_79727/1759186076.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.u_result.append(float(self.u_n_m[i]))


,x,u аналит.,u числ.,delta u
0,0.000000E+00,6.931472E-01,6.931472E-01,0.000000E+00
1,1.000000E-01,6.931603E-01,6.931602E-01,2.310793E-08
2,2.000000E-01,6.933656E-01,6.933655E-01,2.568935E-08
3,3.000000E-01,6.942963E-01,6.942963E-01,2.584237E-08
4,4.000000E-01,6.969030E-01,6.969030E-01,2.172254E-08
5,5.000000E-01,7.025691E-01,7.025691E-01,1.126062E-08
6,6.000000E-01,7.130625E-01,7.130625E-01,6.960143E-09
7,7.000000E-01,7.303954E-01,7.303954E-01,3.230314E-08
8,8.000000E-01,7.565768E-01,7.565769E-01,6.083271E-08
9,9.000000E-01,7.932772E-01,7.932773E-01,8.554652E-08


9.90E-08
201


/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_79727/1759186076.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.u_result.append(float(self.u_n_m[i]))


,x,u аналит.,u числ.,delta u
0,0.000000E+00,6.931472E-01,6.931472E-01,0.000000E+00
1,1.000000E-01,6.931603E-01,6.931603E-01,1.448658E-09
2,2.000000E-01,6.933656E-01,6.933656E-01,1.564516E-09
3,3.000000E-01,6.942963E-01,6.942963E-01,1.495006E-09
4,4.000000E-01,6.969030E-01,6.969030E-01,1.123228E-09
5,5.000000E-01,7.025691E-01,7.025691E-01,3.214135E-10
6,6.000000E-01,7.130625E-01,7.130625E-01,9.911308E-10
7,7.000000E-01,7.303954E-01,7.303954E-01,2.759043E-09
8,8.000000E-01,7.565768E-01,7.565768E-01,4.717081E-09
9,9.000000E-01,7.932772E-01,7.932772E-01,6.415583E-09


7.40E-09
401


/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_79727/1759186076.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.u_result.append(float(self.u_n_m[i]))


,x,u аналит.,u числ.,delta u
0,0.000000E+00,6.931472E-01,6.931472E-01,0.000000E+00
1,1.000000E-01,6.931603E-01,6.931603E-01,8.945100E-11
2,2.000000E-01,6.933656E-01,6.933656E-01,9.161172E-11
3,3.000000E-01,6.942963E-01,6.942963E-01,7.874046E-11
4,4.000000E-01,6.969030E-01,6.969030E-01,4.354139E-11
5,5.000000E-01,7.025691E-01,7.025691E-01,2.182354E-11
6,6.000000E-01,7.130625E-01,7.130625E-01,1.220344E-10
7,7.000000E-01,7.303954E-01,7.303954E-01,2.530117E-10
8,8.000000E-01,7.565768E-01,7.565768E-01,3.974162E-10
9,9.000000E-01,7.932772E-01,7.932772E-01,5.266406E-10


6.12E-10
